In [133]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [134]:
# Set the data directory and sequence length
data_directory = "../data/filtered_blocks_padded/"
sequence_length = 36
batch_size = 32  # Adjust as needed

In [135]:
# Encoding legend
encoding_legend = {
    1: 'MRI_CCS_11', 2: 'MRI_EXU_95', 3: 'MRI_FRR_18', 4: 'MRI_FRR_257',
    5: 'MRI_FRR_264', 6: 'MRI_FRR_3', 7: 'MRI_FRR_34', 8: 'MRI_MPT_1005',
    9: 'MRI_MSR_100', 10: 'START', 11: 'MRI_MSR_21', 12: 'MRI_MSR_34',
    0: 'PADDED',  # Add a padding category
    10: 'START',  # Start token
    9: 'END'      # End token
}

In [136]:
def load_data(directory, sequence_length):
    inputs_list = []
    targets_list = []
    file_count = 0  # Track the number of files processed

    for file_name in os.listdir(directory):
        if file_name.endswith('.csv'):
            file_count += 1
            file_path = os.path.join(directory, file_name)
            df = pd.read_csv(file_path)
            
            inputs = df['sourceID'].values
            targets = df['sourceID'].shift(-1).fillna(0).values  # Shift by 1 and replace NaN with 0

            inputs = inputs.astype(int)
            targets = targets.astype(int)

            input_sequences = []
            target_sequences = []

            # Pad sequences shorter than sequence_length
            for i in range(len(inputs) - sequence_length + 1):
                input_sequences.append(inputs[i:i + sequence_length])
                target_sequences.append(targets[i:i + sequence_length])

            inputs_list.append(np.array(input_sequences))
            targets_list.append(np.array(target_sequences))
    
    # Combine all the sequences from all blocks
    inputs = np.concatenate(inputs_list, axis=0)
    targets = np.concatenate(targets_list, axis=0)

    print(f"Total files processed: {file_count}")
    print(f"Combined inputs shape: {inputs.shape}")
    print(f"Combined targets shape: {targets.shape}")

    return inputs, targets


In [137]:
# Load data from the directory
inputs, targets = load_data(data_directory, sequence_length)
print(f"Loaded dataset with inputs shape {inputs.shape} and targets shape {targets.shape}")

Total files processed: 326
Combined inputs shape: (326, 36)
Combined targets shape: (326, 36)
Loaded dataset with inputs shape (326, 36) and targets shape (326, 36)


In [138]:
# Create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((inputs, targets))
dataset = dataset.batch(batch_size, drop_remainder=False)

In [139]:
# Transformer Model Implementation
class PositionalEncoding(layers.Layer):
    def __init__(self, sequence_length, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(sequence_length, d_model)
    
    def get_angles(self, pos, i, d_model):
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
        return pos * angle_rates

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(np.arange(position)[:, np.newaxis], np.arange(d_model)[np.newaxis, :], d_model)

        # Apply sine to even indices in the array
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

        # Apply cosine to odd indices in the array
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        pos_encoding = angle_rads[np.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [140]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [141]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [142]:
def create_transformer_model(num_classes, sequence_length, embedding_dim=64, num_heads=4, ff_dim=128, num_blocks=4):
    print(f"Creating Transformer Model with:")
    print(f"- Vocabulary size: {num_classes}")
    print(f"- Sequence length: {sequence_length}")
    print(f"- Embedding dimension: {embedding_dim}")
    print(f"- Number of heads: {num_heads}")
    print(f"- FFN dimension: {ff_dim}")
    print(f"- Number of blocks: {num_blocks}")

    inputs = tf.keras.Input(shape=(sequence_length,))
    embedding = tf.keras.layers.Embedding(num_classes, embedding_dim)(inputs)
    
    x = embedding
    for _ in range(num_blocks):
        attention_output = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(x, x)
        attention_output = tf.keras.layers.LayerNormalization()(attention_output + x)
        
        ffn_output = tf.keras.layers.Dense(ff_dim, activation='relu')(attention_output)
        ffn_output = tf.keras.layers.Dense(embedding_dim)(ffn_output)
        x = tf.keras.layers.LayerNormalization()(ffn_output + attention_output)
    
    output = tf.keras.layers.Dense(num_classes)(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model


In [143]:
# Parameters for the model
vocab_size = np.max(inputs) + 1  # Number of unique sourceIDs
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

In [144]:
# Create the model
model = create_transformer_model(vocab_size, sequence_length, embed_dim, num_heads, ff_dim)
model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
model.summary()

Creating Transformer Model with:
- Vocabulary size: 13
- Sequence length: 36
- Embedding dimension: 32
- Number of heads: 2
- FFN dimension: 32
- Number of blocks: 4


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape             ┃          Param # ┃ Connected to              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)   │ (None, 36)               │                0 │ -                         │
├───────────────────────────────┼──────────────────────────┼──────────────────┼───────────────────────────┤
│ embedding_15 (Embedding)      │ (None, 36, 32)           │              416 │ input_layer_13[0][0]      │
├───────────────────────────────┼──────────────────────────┼──────────────────┼───────────────────────────┤
│ multi_head_attention_18       │ (None, 36, 32)           │            8,416 │ embedding_15[0][0],       │
│ (MultiHeadAttention)          │                          │                  │ embedding_15[0][0]        │
├───────────────────────────────┼──────────────────────────┼──────────────────┼───────────────────────────┤
│ add_24 (Add)                  │ (None, 36, 32)           │                0 │ multi_head_attention_18[… │
│                               │                          │                  │ embedding_15[0][0]        │
├───────────────────────────────┼──────────────────────────┼──────────────────┼───────────────────────────┤
│ layer_normalization_36        │ (None, 36, 32)           │               64 │ add_24[0][0]              │
│ (LayerNormalization)          │                          │                  │                           │
├───────────────────────────────┼──────────────────────────┼──────────────────┼───────────────────────────┤
│ dense_47 (Dense)              │ (None, 36, 32)           │            1,056 │ layer_normalization_36[0… │
├───────────────────────────────┼──────────────────────────┼──────────────────┼───────────────────────────┤
│ dense_48 (Dense)              │ (None, 36, 32)           │            1,056 │ dense_47[0][0]            │
├───────────────────────────────┼──────────────────────────┼──────────────────┼───────────────────────────┤
│ add_25 (Add)                  │ (None, 36, 32)           │                0 │ dense_48[0][0],           │
│                               │                          │                  │ layer_normalization_36[0… │
├───────────────────────────────┼──────────────────────────┼──────────────────┼───────────────────────────┤
│ layer_normalization_37        │ (None, 36, 32)           │               64 │ add_25[0][0]              │
│ (LayerNormalization)          │                          │                  │                           │
├───────────────────────────────┼──────────────────────────┼──────────────────┼───────────────────────────┤
│ multi_head_attention_19       │ (None, 36, 32)           │            8,416 │ layer_normalization_37[0… │
│ (MultiHeadAttention)          │                          │                  │ layer_normalization_37[0… │
├───────────────────────────────┼──────────────────────────┼──────────────────┼───────────────────────────┤
│ add_26 (Add)                  │ (None, 36, 32)           │                0 │ multi_head_attention_19[… │
│                               │                          │                  │ layer_normalization_37[0… │
├───────────────────────────────┼──────────────────────────┼──────────────────┼───────────────────────────┤
│ layer_normalization_38        │ (None, 36, 32)           │               64 │ add_26[0][0]              │
│ (LayerNormalization)          │                          │                  │                           │
├───────────────────────────────┼──────────────────────────┼──────────────────┼───────────────────────────┤
│ dense_49 (Dense)              │ (None, 36, 32)           │            1,056 │ layer_normalization_38[0… │
├───────────────────────────────┼──────────────────────────┼──────────────────┼───────────────────────────┤
│ de

 Total params: 43,469 (169.80 KB)

 Trainable params: 43,469 (169.80 KB)

 Non-trainable params: 0 (0.00 B)

In [145]:
# Training the model
epochs = 10
for batch in dataset.take(1):  # Take one batch as an example
    input_batch, target_batch = batch
    print(f"Example batch - Input shape: {input_batch.shape}, Target shape: {target_batch.shape}")

history = model.fit(dataset, epochs=epochs)


Example batch - Input shape: (32, 36), Target shape: (32, 36)
Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 23s 35ms/step - accuracy: 0.5115 - loss: 1.7440
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.8528 - loss: 0.5159
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.8564 - loss: 0.4419
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8583 - loss: 0.4169
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.8583 - loss: 0.4024
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.8585 - loss: 0.3935
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.8612 - loss: 0.3873
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.8613 - loss: 0.3820
Epoch 9/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.8635 - loss: 0.3771
Epoch 10/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.8640 - loss: 0.3712


In [146]:
# Mapping predictions back to human-readable labels
def map_prediction_to_label(prediction, encoding_legend):
    mapped_labels = []
    for token in prediction.flatten():  # Flatten to handle each token individually
        label = encoding_legend.get(token, "Unknown")  # Retrieve label or mark as "Unknown"
        mapped_labels.append(label)
    return mapped_labels

In [147]:
def predict_next_sequence(input_sequence, model):
    print(f"Input sequence for prediction: {input_sequence}")
    input_sequence = np.array(input_sequence).reshape(1, -1)  # Reshape for single input
    prediction = model.predict(input_sequence)
    
    print(f"Prediction raw output shape: {prediction.shape}")
    
    predicted_sequence = np.argmax(prediction, axis=-1)  # Get predicted token (sourceID)
    print(f"Predicted sequence (encoded): {predicted_sequence}")
    
    # Mapping to human-readable labels
    human_readable_prediction = map_prediction_to_label(predicted_sequence[0], encoding_legend)
    print(f"Predicted sequence (human-readable): {human_readable_prediction}")
    
    return predicted_sequence, human_readable_prediction


In [148]:
# Example prediction
example_input = inputs[0]  # Use the first input sequence as an example
predicted_sequence, human_readable_sequence = predict_next_sequence(example_input, model)
print(f"Predicted sequence (encoded): {predicted_sequence}")
print(f"Predicted sequence (human-readable): {human_readable_sequence}")

Input sequence for prediction: [10  4  5  1 12  5  8  4  5  5  1  5  4  5  9  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 901ms/step
Prediction raw output shape: (1, 36, 13)
Predicted sequence (encoded): [[4 5 1 1 8 1 9 5 1 1 1 1 5 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Predicted sequence (human-readable): ['MRI_FRR_257', 'MRI_FRR_264', 'MRI_CCS_11', 'MRI_CCS_11', 'MRI_MPT_1005', 'MRI_CCS_11', 'END', 'MRI_FRR_264', 'MRI_CCS_11', 'MRI_CCS_11', 'MRI_CCS_11', 'MRI_CCS_11', 'MRI_FRR_264', 'MRI_CCS_11', 'PADDED', 'PADDED', 'PADDED', 'PADDED', 'PADDED', 'PADDED', 'PADDED', 'PADDED', 'PADDED', 'PADDED', 'PADDED', 'PADDED', 'PADDED', 'PADDED', 'PADDED', 'PADDED', 'PADDED', 'PADDED', 'PADDED', 'PADDED', 'PADDED', 'PADDED']
Predicted sequence (encoded): [[4 5 1 1 8 1 9 5 1 1 1 1 5 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Predicted sequence (human-readable): ['MRI_FRR_257', 'MRI_FRR_264', 'MRI_CCS_11', 'MRI_CCS_11', 'MRI_MPT_1005', '